## Aprendizado Supervisionado
### prof. Duncan
Prática com KNN e Iris

In [ ]:
#pacote para profiling de datasets
#!pip install ydata_profiling
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from ydata_profiling import ProfileReport


# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
#
# pacote para separação entre treino e teste
from sklearn.model_selection import train_test_split
#
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
#
# pacotes para validação cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
#
# pacotes de indução de modelos
from sklearn.neighbors import KNeighborsClassifier
#
# pacotes para avaliação dos resultados
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.tree import plot_tree, export_graphviz, export_text

#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv

#pacotes para visualização e formatação
import pprint
import graphviz
#
# configurações para os diferentes pacotes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# carga de dados
iris = datasets.load_iris(as_frame=True)
print(iris.DESCR)

In [ ]:
# separação em features e target e profiling do dataset
X = iris.data
y = iris.target
Xy = iris.frame
#relatorio = ProfileReport(Xy, title="Iris data set")
#relatorio.to_notebook_iframe()


In [ ]:
Xy

In [ ]:
# procedimento para listar treino e teste
def lista_treino_teste(treino_X, treino_y, teste_X, teste_y):
  treino = treino_X.copy()
  treino.set_axis(['sepallength', 'sepalwidth', 'petallength', 'petalwidth'], axis="columns", copy=False)
  treino['class'] = treino_y.copy()
  teste = teste_X.copy()
  teste.set_axis(['sepallength', 'sepalwidth', 'petallength', 'petalwidth'], axis="columns", copy=False)
  teste['class'] = teste_y.copy()
  treinoteste = pd.concat([treino, teste], ignore_index=True)
  print(treino_X.shape)
  print(treino_y.shape)
  print(teste_X.shape)
  print(teste_y.shape)
  print(np.stack(np.unique(teste_y, return_counts=True), axis=1))
  print(treinoteste.to_string())


In [ ]:
# separação em treino e teste, e X e y
# importante fazer ANTES do pipeline para evitar data leakage

tr_unsh_X, te_unsh_X, tr_unsh_y, te_unsh_y = train_test_split(X, y, random_state=0,test_size=0.333, shuffle=False)

treino_X, teste_X, treino_y, teste_y = train_test_split(X, y, random_state=46,test_size=0.333, stratify=y)


In [ ]:
lista_treino_teste(tr_unsh_X, tr_unsh_y, te_unsh_X, te_unsh_y)

In [ ]:
lista_treino_teste(treino_X, treino_y, teste_X, teste_y)

In [ ]:
# configurações para experimentos

n_vizinhos = [1, 3, 5, 7, 9, 11, 13]
#n_vizinhos = [1, 3, 5]

k_splits = 10


## Experimento sem reescala de valores e sem embaralhamento dos dados

In [ ]:
# indução do modelo para diferentes números de vizinhos

for n in n_vizinhos:
    modelo = KNeighborsClassifier(n_neighbors=n)
    modelo.fit(tr_unsh_X, tr_unsh_y)
    te_unsh_pred_y = modelo.predict(te_unsh_X)
    acuracia = accuracy_score(te_unsh_y, te_unsh_pred_y)
    resultado = confusion_matrix(te_unsh_y, te_unsh_pred_y)
    confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
    fig = confusion_matrix_display.figure_
    fig.set_figheight(3)
    fig.set_figwidth(3)
    print('k=', n,'  Acuracia=', acuracia)


## Experimento sem reescala de valores e com embaralhamento dos dados

In [ ]:
# indução do modelo para diferentes números de vizinhos

for n in n_vizinhos:
    modelo = KNeighborsClassifier(n_neighbors=n)
    modelo.fit(treino_X, treino_y)
    teste_pred_y = modelo.predict(teste_X)
    acuracia = accuracy_score(teste_y, teste_pred_y)
    resultado = confusion_matrix(teste_y, teste_pred_y)
    confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
    fig = confusion_matrix_display.figure_
    fig.set_figheight(3)
    fig.set_figwidth(3)
    print('k=', n,'  Acuracia=', acuracia)



### Modelagem usando validação cruzada e com indução explícita para cada pasta

In [ ]:
%%time
# indução do modelo para diferentes números de vizinhos com validação cruzada

for n in n_vizinhos:
    acuracia = np.zeros(k_splits)
    modelo = KNeighborsClassifier(n_neighbors=n)
#    kf = KFold(n_splits=k_splits, random_state=None, shuffle=False)
    kf = StratifiedKFold(n_splits=k_splits, random_state=46,shuffle=True)
    print('k={} ------------------------'.format(n))
    for i, (tr_index, te_index) in enumerate(kf.split(X, y)):
      tr_X = X.iloc[tr_index]
      te_X = X.iloc[te_index]
      tr_y = np.ravel(y.iloc[tr_index])
      te_y = np.ravel(y.iloc[te_index])
      modelo.fit(tr_X, tr_y)
      te_pred_y = modelo.predict(te_X)
      acuracia[i] = accuracy_score(te_y, te_pred_y)
      print('k={} i={} Acuracia={:.3f}'.format(n, i, acuracia[i]))
      resultado = confusion_matrix(te_y, te_pred_y)
      confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
      fig = confusion_matrix_display.figure_
      fig.set_figheight(3)
      fig.set_figwidth(3)
    print('k={}  Acuracia={:.3f}'.format(n, np.mean(acuracia)))



In [ ]:
print(list(enumerate(kf.split(X, y))))

## Modelagem com validação cruzada e com busca exaustiva pelos parâmetros

In [ ]:
# indução do modelo para diferentes números de vizinhos com validação cruzada

parametros = { 'n_neighbors': n_vizinhos, 'weights':['uniform', 'distance']}
modelo = KNeighborsClassifier()
valcruz = GridSearchCV(modelo, parametros,cv=k_splits)
valcruz.fit(X, y)
resultados = pd.DataFrame(valcruz.cv_results_).sort_values(by='rank_test_score')
print('Melhor resultado - Número vizinhos:{}   Pesos:{}    Acurácia média:{:.3f}'.format(
    resultados['param_n_neighbors'].iloc[0],
    resultados['param_weights'].iloc[0],
    resultados['mean_test_score'].iloc[0]))


In [ ]:
resultados

In [ ]:
valcruz.cv_results_['params']

## Experimentos com reescala de valores
### Preparação e modelagem, cuidando para não ocorrer data leakage

### Experimento sem embaralhamento dos dados


In [ ]:
# protocolo experimental versão

for n in n_vizinhos:
  escalonador = MinMaxScaler()
#  escalonador = StandardScaler()
  estimador = KNeighborsClassifier(n_neighbors= n)
  tr_unsh_X_escalonado = escalonador.fit_transform(tr_unsh_X, tr_unsh_y)
  estimador.fit(tr_unsh_X_escalonado, tr_unsh_y)
  te_unsh_X_escalonado = escalonador.transform(te_unsh_X)
  te_unsh_pred_y = estimador.predict(te_unsh_X_escalonado)
  acuracia = accuracy_score(te_unsh_y, te_unsh_pred_y)
  resultado = confusion_matrix(te_unsh_y, te_unsh_pred_y)
  confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
  fig = confusion_matrix_display.figure_
  fig.set_figheight(3)
  fig.set_figwidth(3)
  print('k={}  Acuracia={:.3f}'.format(n, acuracia))

### Experimento com reescala e embaralhamento dos dados

In [ ]:
# protocolo experimental

for n in n_vizinhos:
  escalonador = MinMaxScaler()
#  escalonador = StandardScaler()
  estimador = KNeighborsClassifier(n_neighbors= n)
  treino_X_escalonado = escalonador.fit_transform(treino_X, treino_y)
  estimador.fit(treino_X_escalonado, treino_y)
  teste_X_escalonado = escalonador.transform(teste_X)
  teste_pred_y = estimador.predict(teste_X_escalonado)
  acuracia = accuracy_score(teste_y, teste_pred_y)
  resultado = confusion_matrix(teste_y, teste_pred_y)
  confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
  fig = confusion_matrix_display.figure_
  fig.set_figheight(3)
  fig.set_figwidth(3)
  print('k={}  Acuracia={:.3f}'.format(n, acuracia))

In [ ]:
# conteúdo de X e y com reescala de valores

treino_escalonado = pd.DataFrame({'sepallength': treino_X_escalonado[:, 0],
                                  'sepalwidth': treino_X_escalonado[:, 1],
                                  'petallength': treino_X_escalonado[:, 2],
                                  'petalwidth': treino_X_escalonado[:, 3],
                                  'class': treino_y})

teste_escalonado = pd.DataFrame({'sepallength': teste_X_escalonado[:, 0],
                                  'sepalwidth': teste_X_escalonado[:, 1],
                                  'petallength': teste_X_escalonado[:, 2],
                                  'petalwidth': teste_X_escalonado[:, 3],
                                  'class': teste_y})

In [ ]:
iris_escalonado = pd.concat([treino_escalonado, teste_escalonado], ignore_index=True)
print(iris_escalonado.to_string())

In [ ]:
iris_escalonado.dtypes


## Experimento com reescala de valores, embaralhamento e busca exaustiva nos parâmetros

In [ ]:
# protocolo experimental com preparação e modelagem
modelo = Pipeline(steps=[
    ('reescala', MinMaxScaler()),
     ('modelagem', KNeighborsClassifier())
    ])
parametros = {'modelagem__n_neighbors':n_vizinhos}
kfold = KFold(n_splits=k_splits, shuffle=True, random_state=0)
grade = GridSearchCV(modelo, param_grid=parametros, cv=kfold)
grade.fit(X,y)
print('Melhor k:{}  score:{:0.3f}'.format(grade.best_params_, grade.best_score_))

In [ ]:
grade

In [ ]:
resultados = pd.DataFrame(grade.cv_results_).sort_values(by='rank_test_score')
resultados

### Captura dos melhores parâmetros para indução do modelo de aprendizado

In [ ]:
# captura da melhor configuração
n_neighbors_best = grade.best_params_['modelagem__n_neighbors']
escalonador = MinMaxScaler()
modelo = KNeighborsClassifier(n_neighbors=n_neighbors_best)
treino_X_escalonado = escalonador.fit_transform(treino_X, treino_y)
teste_X_escalonado = escalonador.transform(teste_X)
modelo.fit(treino_X_escalonado, treino_y)
teste_pred_y = modelo.predict(teste_X_escalonado)
acuracia = accuracy_score(teste_y, teste_pred_y)
resultado = confusion_matrix(teste_y, teste_pred_y)
confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
fig = confusion_matrix_display.figure_
fig.set_figheight(3)
fig.set_figwidth(3)
print('k=', n_neighbors_best,'  Acuracia=', acuracia)



## Mostra da distribuição de pontos e classes, para petallength e petalwidth, no treino

In [ ]:
X_mostra = treino_X_escalonado[:, 2:4].copy()
modelo_mostra = KNeighborsClassifier(n_neighbors=n_neighbors_best)
modelo_mostra.fit(X_mostra, treino_y)
mostra = DecisionBoundaryDisplay.from_estimator(
    modelo_mostra,
    X_mostra,
    response_method="predict",
    cmap=plt.cm.Set3,
    xlabel='petallength',
    ylabel='petalwidth',
)
mostra.ax_.scatter(
    X_mostra[:, 0], X_mostra[:, 1], c=treino_y, s=50,
    cmap=plt.cm.viridis, edgecolor="black", linewidth=1
)

## Agora, no teste

In [ ]:
X_mostra = teste_X_escalonado[:, 2:4].copy()
mostra = DecisionBoundaryDisplay.from_estimator(
    modelo_mostra,
    X_mostra,
    response_method="predict",
    cmap=plt.cm.Set3,
    xlabel='petallength',
    ylabel='petalwidth',
)
mostra.ax_.scatter(
    X_mostra[:, 0], X_mostra[:, 1], c=teste_y, s=50,
    cmap=plt.cm.viridis, edgecolor="black", linewidth=1
)